In [54]:
# Dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
import json
from census import Census

In [6]:
# Census API key
from courtCensusKey import censusKey

In [16]:
complete_url='https://api.census.gov/data/timeseries/intltrade/exports/hs'+\
            '?get=DISTRICT,DIST_NAME,ALL_VAL_MO,ALL_VAL_YR,E_COMMODITY_SDESC'+\
            '&E_COMMODITY=3926201010'+\
            '&YEAR=2019'+\
            '&MONTH=*'

In [17]:
response=requests.get(complete_url)
response

<Response [200]>

In [18]:
pd.DataFrame(response.json())

,0,1,2,3,4,5,6,7
0,DISTRICT,DIST_NAME,ALL_VAL_MO,ALL_VAL_YR,E_COMMODITY_SDESC,E_COMMODITY,YEAR,MONTH
1,-,TOTAL FOR ALL DISTRICTS,1109949,1109949,"GLOVES,SEAMLESS,SURGICAL & MEDICAL,OF PLASTIC",3926201010,2019,01
2,07,"OGDENSBURG, NY",99903,99903,"GLOVES,SEAMLESS,SURGICAL & MEDICAL,OF PLASTIC",3926201010,2019,01
3,17,"SAVANNAH, GA",3380,3380,"GLOVES,SEAMLESS,SURGICAL & MEDICAL,OF PLASTIC",3926201010,2019,01
4,09,"BUFFALO, NY",237669,237669,"GLOVES,SEAMLESS,SURGICAL & MEDICAL,OF PLASTIC",3926201010,2019,01
...,...,...,...,...,...,...,...,...
330,49,"SAN JUAN, PR",72819,519368,"GLOVES,SEAMLESS,SURGICAL & MEDICAL,OF PLASTIC",3926201010,2019,12
331,52,"MIAMI, FL",67122,1557770,"GLOVES,SEAMLESS,SURGICAL & MEDICAL,OF PLASTIC",3926201010,2019,12
332,53,"HOUSTON-GALVESTON, TX",0,39082,"GLOVES,SEAMLESS,SURGICAL & MEDICAL,OF PLASTIC",3926201010,2019,12
333,55,"DALLAS-FORT WORTH, TX",0,82464,"GLOVES,SEAMLESS,SURGICAL & MEDICAL,OF PLASTIC",3926201010,2019,12


In [28]:
#Setting variables 
year='2019'
month='*'
cols='DISTRICT,DIST_NAME,ALL_VAL_MO,ALL_VAL_YR,E_COMMODITY_SDESC'


In [99]:
#Base URL 
base_url = 'https://api.census.gov/data/timeseries/intltrade/exports/hs'

complete_url = f'{base_url}?get={cols}&YEAR={year}&MONTH={month}'


hts_list=['2926100000','4002510000','4002590000','2836500000','4002110000','3904100000']
df_list=[]

for each_hts in hts_list:
    hts_code=each_hts
    query_params={
        'get': 'DISTRICT,DIST_NAME,ALL_VAL_MO,ALL_VAL_YR,E_COMMODITY_SDESC',
        'E_COMMODITY': hts_code,
        'YEAR': '2019',
        'MONTH': "*"
    }
    max_response=requests.get(base_url, params=query_params)
    #print(max_response.json())
    df_list.append(max_response.json()[1:])
#print(df_list)

In [98]:
productList = []
for product in df_list:
    productList += product

In [96]:
exportData = pd.DataFrame(productList, columns=max_response.json()[0])
exportData

,DISTRICT,DIST_NAME,ALL_VAL_MO,ALL_VAL_YR,E_COMMODITY_SDESC,E_COMMODITY,YEAR,MONTH
0,07,"OGDENSBURG, NY",267236,267236,ACRYLONITRILE,2926100000,2019,01
1,20,"NEW ORLEANS, LA",2496000,2496000,ACRYLONITRILE,2926100000,2019,01
2,23,"LAREDO, TX",41529,41529,ACRYLONITRILE,2926100000,2019,01
3,38,"DETROIT, MI",71346,71346,ACRYLONITRILE,2926100000,2019,01
4,53,"HOUSTON-GALVESTON, TX",65868657,65868657,ACRYLONITRILE,2926100000,2019,01
...,...,...,...,...,...,...,...,...
1658,52,"MIAMI, FL",185354,3129864,"POLYVINYL CHLORIDE,NOT MIXED WITH ANY OTH SUBS...",3904100000,2019,12
1659,53,"HOUSTON-GALVESTON, TX",80743580,1177505442,"POLYVINYL CHLORIDE,NOT MIXED WITH ANY OTH SUBS...",3904100000,2019,12
1660,54,"WASHINGTON, DC",0,3240,"POLYVINYL CHLORIDE,NOT MIXED WITH ANY OTH SUBS...",3904100000,2019,12
1661,70,LOW VALUE,4166,155238,"POLYVINYL CHLORIDE,NOT MIXED WITH ANY OTH SUBS...",3904100000,2019,12


In [100]:
exportData['E_COMMODITY_SDESC'].unique()

array(['ACRYLONITRILE', 'LATEX OF ACRYLONITRILE-BUTADIENE RUBBER (NBR)',
       'ACRYLONITRILE-BUTADIENE RUBBER, EXC LATEX, NESOI',
       'CALCIUM CARBONATE', 'LATEX OF (SBR); (XSBR)',
       'POLYVINYL CHLORIDE,NOT MIXED WITH ANY OTH SUBSTANC'], dtype=object)

In [ ]:
data_url = f'{base_url}?get={cols}&for=county:{county}&in=state:{state}'
data_url

In [ ]:
#Retrieve Data
response=requests.get(data_url)
print(response.text)

In [ ]:
#Convert to county data to dataframe, perform some basic operations
county_df=pd.DataFrame(countyData[1:], columns=countyData[0]).\
rename(columns={"GEO_ID": "GEOID", "GEONAME": "CountyST"})
county_df['fips']=county_df.state+county_df.county
county_df.set_index('fips',inplace=True)
county_df.drop(columns=['state','county'],inplace=True)
county_df[['County','State']] = county_df.CountyST.str.split(",",expand=True)
county_df.dropna(how="any", inplace=True)
county_df